<a href="https://colab.research.google.com/github/rayrui312/ccxtfinrl_btc/blob/main/demo_stock_wrds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Stock Trading Using WRDS Data in NeoFinRL**






# **Part 1: Install NeoFinRL, ElegantRL and related packages**

In [38]:
!pip install git+https://github.com/AI4Finance-LLC/NeoFinRL.git
!pip install git+https://github.com/AI4Finance-LLC/ElegantRL.git
!pip install yfinance stockstats
!pip install wrds
!pip install trading_calendars

  Cloning https://github.com/AI4Finance-LLC/NeoFinRL.git to /tmp/pip-req-build-x7u_ss8i
  Running command git clone -q https://github.com/AI4Finance-LLC/NeoFinRL.git /tmp/pip-req-build-x7u_ss8i
  Created wheel for neofinrl: filename=neofinrl-0.0.1-cp37-none-any.whl size=70711 sha256=74d00eac524c2f5c1f67fe690bf947886213a4fd92db80ce3a054cb11a683a63
  Stored in directory: /tmp/pip-ephem-wheel-cache-mveahk3u/wheels/fa/c0/c9/bf3369bebc879bb6d0f2fbbe67715776c0dc1302dfa94ccd8f
Successfully built neofinrl
  Cloning https://github.com/AI4Finance-LLC/ElegantRL.git to /tmp/pip-req-build-4notceg9
  Running command git clone -q https://github.com/AI4Finance-LLC/ElegantRL.git /tmp/pip-req-build-4notceg9
  Created wheel for elegantrl: filename=elegantrl-0.3.1-cp37-none-any.whl size=65369 sha256=c57de19d4a38112bfa9fc83e699df18ff80981c3c79abe452dd128607f074f4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-y55ovncl/wheels/d0/f4/2e/cec0c14b57c2094a2bcef3063f95d758ad1309a640ff100419
Successfully built

# **Part 2: Import Packages**


*   **NeoFinRL**
*   **ElegantRL**



In [39]:
from elegantrl.run import *
from neo_finrl.wrds.wrds_engineer import WrdsEngineer
from neo_finrl.wrds.env_stock_wrds import StockTradingEnv
from elegantrl.agent import *

# **Part 3: Data Fetching and Pre-processing**

*   **WrdsEngineer**: An aggregation class of data fetching and pre-processing for WRDS database
*   **WrdsEngineer.data_fetch_ohlcv**: Download data by sending sql query to WRDS and preprocessing the raw data to get ohlcv (open, high, low, close, volume) dataframe.
*   **WrdsEngineer.data_clean**: Clean the ohlcv data. (Fill up missing rows and missing values.)
*   **WrdsEngineer.add_technical_indicators**: Add technical indicators to cleaned ohlcv data.
*   **WrdsEngineer.df_to_ary**: Transform the fully preprocessed dataframe into numpy arrary (to put into the environment).



> After calling this methods step by step, we will finally get a numpy array to directly put into our environment.




In [ ]:
# Data Fetching
#DOW_30_TICKER
stock_list = ["AAPL","MSFT","JPM","V","RTX","PG","GS","NKE","DIS","AXP","HD",
        "INTC","WMT","IBM","MRK","UNH","KO","CAT","TRV","JNJ","CVX",
        "MCD","VZ","CSCO","XOM","BA","MMM","PFE","WBA","DD"]
tech_indicator_list = ['macd','boll_ub','boll_lb','rsi_30','dx_30',
                       'close_30_sma','close_60_sma']
# initialize WrdsEngineer, please fill in your own account info
WE = WrdsEngineer() 
# fetch raw data and calculate ohlcv 
df = WE.data_fetch_ohlcv(start='2021-01-01',end='2021-05-01',stock_list=stock_list, time_interval=60,
                         if_save_tempfile=False)
"""The raw data from WRDS TAQ database is very large. The step above may take hours!!!
Here we strongly recommend you save the temp file locally by setting 
if_save_tempfile=True."""
# clean the ohlcv data
df = WE.data_clean(df)
# add technical indicators
df = WE.add_technical_indicators(df,tech_indicator_list=tech_indicator_list)
print(df.head())
# transform dataframe to numpy array
ary = WE.df_to_ary(df, tech_indicator_list)
""" You can also save the final array file locally by adding
np.save('xxx',ary)"""
print(ary[:10])


# **Part 4: Train, Evaluate and Backtest the Agent by ElegantRL**

See https://github.com/AI4Finance-LLC/ElegantRL/blob/master/eRL_demo_StockTrading.ipynb for more demonstrations.






In [ ]:
args = Arguments(if_on_policy=True)
args.agent = AgentPPO()

#choose environment
args.env = StockTradingEnv(ary = ary, stock_dim = 30, if_train=True)
args.env_eval = StockTradingEnv(ary = ary, stock_dim = 30, if_train=False)
args.net_dim = 2 ** 9 # change a default hyper-parameters
args.batch_size = 2 ** 8
args.break_step = int(5e5)

train_and_evaluate(args)

env = StockTradingEnv(ary=ary, stock_dim=30 ,if_train=False)
args = Arguments(if_on_policy=True)
args.agent = AgentPPO()
args.env = StockTradingEnv(ary=ary, stock_dim = 30, if_train=False)
args.if_remove = False
args.cwd = './AgentPPO/Stock_wrds-v1_0'
args.init_before_training()

env.draw_cumulative_return(args, torch)